In [ ]:
import holoviews as hv

hv.extension("bokeh")
import geoviews as gv

import numpy as np
from glob import glob
from time import time
from importlib import reload
import os, sys
from slcl1butils.get_polygons_from_l1b import get_swath_tiles_polygons_from_l1bfiles

In [ ]:
from slcl1butils.utils import get_test_file

one_safe_l1b = get_test_file(
    "S1B_IW_XSP__1SDV_20210328T055258_20210328T055325_026211_0320D4_DC31_A13.SAFE"
)
l1bncfile_pattern = os.path.join(one_safe_l1b, "*iw*nc")
print(l1bncfile_pattern)
files = glob(l1bncfile_pattern)
len(files)

## Load the polygons pts and corresponding variables from L1B

In [ ]:
import slcl1butils.get_polygons_from_l1b

reload(slcl1butils.get_polygons_from_l1b)
(
    polygons,
    coordinates,
    variables,
) = slcl1butils.get_polygons_from_l1b.get_swath_tiles_polygons_from_l1bfiles(
    files, polarization="VV", variable_names=["sigma0"]
)

## Define boundaries for each variable to adjust the colorbar

In [ ]:
# burst_type = 'intra'
burst_type = "intra"
varis = [var for var in variables["intra"].keys()]
for var in varis:
    variables[burst_type][var + "_clim"] = (
        np.nanpercentile(np.array(variables[burst_type][var]), 1.0),
        np.nanpercentile(np.array(variables[burst_type][var]), 99.0),
    )

## Example for plotting the NRCS (sigma0)

In [ ]:
vari = "sigma0"
variable = variables[burst_type][vari]
clim = variables[burst_type][vari + "_clim"]

#### Build the polygons and associated color for each tiles

In [ ]:
gsigvv = []
for cpt in np.arange(len(variable)):
    pts = [
        (i, j)
        for i, j in zip(
            polygons[burst_type]["tiles"][cpt].exterior.xy[0].tolist(),
            polygons[burst_type]["tiles"][cpt].exterior.xy[1].tolist(),
        )
    ]
    gsigvv.append(
        gv.Polygons(
            {("Longitude", "Latitude"): pts, "level": variable[cpt]},
            vdims="level",
            kdims=["Longitude", "Latitude"],
        ).opts(
            color="NA",
            colorbar=True,
            cmap="Greys_r",
            clim=clim,
            alpha=0.75,
            muted_alpha=0,
        )
    )

#### Build the polygons for tile intra and tiles inter, swath and map background

In [ ]:
# Background cartography
gvvbckg = gv.tile_sources.EsriImagery
# Swath limits
gvvswath = gv.Polygons(polygons["intra"]["swath"], label="swath").opts(
    fill_color="white", alpha=0.25, show_legend=True, muted_alpha=0, tools=["hover"]
)
# Intra and inter bursts polygons
gvvtile_intra = gv.Polygons(polygons["intra"]["tiles"], label="intraburst tiles").opts(
    fill_color="blue", alpha=0.25, show_legend=True, muted_alpha=0, tools=["hover"]
)
gvvtile_inter = gv.Polygons(polygons["inter"]["tiles"], label="interburst tiles").opts(
    fill_color="red", alpha=0.25, show_legend=True, muted_alpha=0
)

#### Simple direct plotting

In [ ]:
(gvvbckg * gvvswath * gvvtile_intra * gvvtile_inter * hv.Overlay(gsigvv)).opts(
    width=1000, height=1500, tools=["hover"], legend_position="top_right"
)